In [1]:
import pandas as pd
import plotly.graph_objects as go
from evento import ajustar_datas

In [2]:
# Fundo sem histórico completo de preços no Yahoo Finance
# Carregar histórico da B3
precos = (
    pd.read_csv('data/knri.csv', parse_dates=['datneg'])
    # a série da B3 começa em "2010-12-01" -> vamos começar em "2011-01-01"
    .query('datneg >= "2011-01-01"')
)
precos

,datneg,preult
20,2011-01-04,100.00
21,2011-01-05,99.70
22,2011-01-06,99.70
23,2011-01-07,99.50
24,2011-01-10,95.50
...,...,...
2993,2023-01-09,139.55
2994,2023-01-10,139.55
2995,2023-01-11,139.97
2996,2023-01-12,139.40


In [3]:
# Histórico de dividendos do HGLG11 extraído do site Funds Explorer
divs = (
    pd.read_csv('data/knri_divs.csv', parse_dates=['data_com', 'data_pag'])
    # a série da B3 começa em "2010-12-01" -> vamos começar em "2011-01-01"
    .query('data_com >= "2011-01-01"')

)
divs

,tipo,data_com,data_pag,dividendo
1,Rendimento,2011-01-31,2011-02-14,0.675
2,Rendimento,2011-02-28,2011-03-16,0.700
3,Rendimento,2011-03-31,2011-04-14,0.700
4,Rendimento,2011-04-29,2011-05-13,0.700
5,Rendimento,2011-05-31,2011-06-14,0.700
...,...,...,...,...
140,Rendimento,2022-08-31,2022-09-15,0.910
141,Rendimento,2022-09-30,2022-10-17,0.910
142,Rendimento,2022-10-31,2022-11-16,0.910
143,Rendimento,2022-11-30,2022-12-14,0.910


In [4]:
# Fazer gráfico de barras com o dividendos pagos pela KNRI11 a cada ano usano o plotly
divs["ano"] = divs["data_com"].dt.year
# Agrupar os dividendos por ano
divs_gb = divs.groupby("ano")["dividendo"].sum().reset_index()
# Criar o gráfico
fig = go.Figure(
    data=[
        go.Bar(
            x=divs_gb["ano"],
            y=divs_gb["dividendo"],
            text=divs_gb["dividendo"].round(1),
            textposition="auto",
        )
    ]
)
fig.update_layout(
    title="Dividendo anual por cota pago pelo KNRI11",
    xaxis_title="Ano",
    yaxis_title="Dividendos (R$)",
    title_x=0.5,
    title_y=0.85,

)
fig.show()

In [5]:
# Houve amortização de capital?
divs["tipo"].value_counts()

Rendimento    144
Name: tipo, dtype: int64

In [6]:
# As colunas "tipo" e "ano" não serão mais usadas
divs.drop(columns=["tipo", "ano"], inplace=True)

In [7]:
# Verificar a quantidade de datas a serem ajustadas
print(len(set(divs['data_com']) - set(ajustar_datas(divs['data_com'], precos['datneg']))), 'datas-com a serem ajustadas')
# Forçar data-com -> datas de negociação do ativo
# anterior=True (padrão) -> se não houver negociação na data-com, ajustar para a primeira data anterior de negociação
divs['data_com'] = ajustar_datas(divs['data_com'], precos['datneg'])
# anterior=False -> se não houver negociação na data de pagamento, ajustar para a primeira data posterior
divs['data_pag'] = ajustar_datas(divs['data_pag'], precos['datneg'], anterior=False)
divs

0 datas-com a serem ajustadas


,data_com,data_pag,dividendo
1,2011-01-31,2011-02-14,0.675
2,2011-02-28,2011-03-16,0.700
3,2011-03-31,2011-04-14,0.700
4,2011-04-29,2011-05-13,0.700
5,2011-05-31,2011-06-14,0.700
...,...,...,...
140,2022-08-31,2022-09-15,0.910
141,2022-09-30,2022-10-17,0.910
142,2022-10-31,2022-11-16,0.910
143,2022-11-30,2022-12-14,0.910


In [8]:
# Verificar se existe alguma "data_com" repetida
divs['data_com'].duplicated().any()

False

In [9]:
# Juntar o histórico de preços com o histórico de dividendos no dataframe "tr" (total return)
# A coluna "data_pag" será usada em etapa posterior
tr = pd.merge(precos, divs[["data_com", "dividendo"]], how="left", left_on="datneg", right_on="data_com")
# Sinalizar que esses dividendos são na data-com -> renomear de "dividendo" para "div_data_com"
tr.rename(columns={'dividendo': 'div_data_com'}, inplace=True)
# Data-com é a data em que existe "dividendo" -> remover coluna "data_com"
tr.drop(columns=["data_com"], inplace=True)
tr

,datneg,preult,div_data_com
0,2011-01-04,100.00,NaN
1,2011-01-05,99.70,NaN
2,2011-01-06,99.70,NaN
3,2011-01-07,99.50,NaN
4,2011-01-10,95.50,NaN
...,...,...,...
2973,2023-01-09,139.55,NaN
2974,2023-01-10,139.55,NaN
2975,2023-01-11,139.97,NaN
2976,2023-01-12,139.40,NaN


In [10]:
# Verificar o início do dataframe "tr" para ver se está tudo certo
tr.loc[15:20]

,datneg,preult,div_data_com
15,2011-01-27,97.00,NaN
16,2011-01-28,96.60,NaN
17,2011-01-31,96.99,0.675
18,2011-02-01,96.99,NaN
19,2011-02-02,99.00,NaN
20,2011-02-03,98.50,NaN


In [11]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["preult"],
        mode="lines",
        name="Preço de fechamento",
        # line=dict(color="#e86f00"),
    )
)
# Adicionar os pontos de dividendos no próprio gráfico de preços
fig.add_trace(
    go.Scatter(
        x=tr.query('div_data_com > 0').datneg,
        y=tr.query('div_data_com > 0')["preult"],
        mode="markers",
        name="Dividendo",
        marker=dict(
            color="black",
            size=3,
            symbol="circle"),
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento do KNRI11<br> (sem ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.19, xanchor="right", x=0.99),
)
fig.show()

In [12]:
# Criar nova coluna "div_data_ex" que é o valor do dividendo na data-ex (ex-dividendos)
tr["div_data_ex"] = tr["div_data_com"].shift(1)
# Só a coluna "div_data_ex" será usada -> remover a coluna "div_data_com"
tr.drop(columns=["div_data_com"], inplace=True)
# Verificar se a data-com foi movida para a data-ex
tr.loc[15:20]

,datneg,preult,div_data_ex
15,2011-01-27,97.00,NaN
16,2011-01-28,96.60,NaN
17,2011-01-31,96.99,NaN
18,2011-02-01,96.99,0.675
19,2011-02-02,99.00,NaN
20,2011-02-03,98.50,NaN


In [13]:
# Inserir o histórico de dividendos na "data_pag" no dataframe "tr" (total return)
tr = tr.merge(divs[["data_pag", "dividendo"]], how="left", left_on="datneg", right_on="data_pag")
# Sinalizar que esses dividendos são na data-pag -> renomear de "dividendo" para "div_data_pag"
tr.rename(columns={'dividendo': 'div_data_pag'}, inplace=True)
# Data-pagm é a data em que existe "div_data_pag" -> remover coluna "data_pag"
tr.drop(columns=["data_pag"], inplace=True)
tr

,datneg,preult,div_data_ex,div_data_pag
0,2011-01-04,100.00,NaN,NaN
1,2011-01-05,99.70,NaN,NaN
2,2011-01-06,99.70,NaN,NaN
3,2011-01-07,99.50,NaN,NaN
4,2011-01-10,95.50,NaN,NaN
...,...,...,...,...
2973,2023-01-09,139.55,NaN,NaN
2974,2023-01-10,139.55,NaN,NaN
2975,2023-01-11,139.97,NaN,NaN
2976,2023-01-12,139.40,NaN,NaN


In [14]:
# Verificar o início do dataframe "tr" para ver se está tudo certo
tr.loc[15:30]

,datneg,preult,div_data_ex,div_data_pag
15,2011-01-27,97.00,NaN,NaN
16,2011-01-28,96.60,NaN,NaN
17,2011-01-31,96.99,NaN,NaN
18,2011-02-01,96.99,0.675,NaN
19,2011-02-02,99.00,NaN,NaN
20,2011-02-03,98.50,NaN,NaN
21,2011-02-04,99.80,NaN,NaN
22,2011-02-07,97.00,NaN,NaN
23,2011-02-08,96.50,NaN,NaN
24,2011-02-09,97.00,NaN,NaN


In [15]:
# Fazer shift de 1 pregão no preço de fechamento para um dos cálculos de retorno
tr["preult_ant"] = tr["preult"].shift(1)
# Mover a coluna "preult_ant" para a posição 2
tr.insert(2, "preult_ant", tr.pop("preult_ant"))
tr.loc[15:30]

,datneg,preult,preult_ant,div_data_ex,div_data_pag
15,2011-01-27,97.00,97.50,NaN,NaN
16,2011-01-28,96.60,97.00,NaN,NaN
17,2011-01-31,96.99,96.60,NaN,NaN
18,2011-02-01,96.99,96.99,0.675,NaN
19,2011-02-02,99.00,96.99,NaN,NaN
20,2011-02-03,98.50,99.00,NaN,NaN
21,2011-02-04,99.80,98.50,NaN,NaN
22,2011-02-07,97.00,99.80,NaN,NaN
23,2011-02-08,96.50,97.00,NaN,NaN
24,2011-02-09,97.00,96.50,NaN,NaN


In [16]:
# Calculando os fatores de reinvestimento de dividendos
# fr_de = 1 + dividendo / (preço de fechamento na data-ex)
# fr_dc = 1 + dividendo / (preço de fechamento na data de pagamento)
# fr_yf (Yahoo Finance) = 1 + dividendo / preço de abertura teórico na data-ex
# fr_yf = 1 + dividendo / (preço de fechamento no dia anterior - dividendo)
tr["fr_de"] = 1 + tr["div_data_ex"] / tr["preult"]
tr["fr_dc"] = 1 + tr["div_data_pag"] / tr["preult"]
tr["fr_yf"] = 1 + tr["div_data_ex"] / (tr["preult_ant"] - tr["div_data_ex"])
# A coluna "preult_ant" não será mais usada -> remover
tr.drop(columns=["preult_ant"], inplace=True)
tr

,datneg,preult,div_data_ex,div_data_pag,fr_de,fr_dc,fr_yf
0,2011-01-04,100.00,NaN,NaN,NaN,NaN,NaN
1,2011-01-05,99.70,NaN,NaN,NaN,NaN,NaN
2,2011-01-06,99.70,NaN,NaN,NaN,NaN,NaN
3,2011-01-07,99.50,NaN,NaN,NaN,NaN,NaN
4,2011-01-10,95.50,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2973,2023-01-09,139.55,NaN,NaN,NaN,NaN,NaN
2974,2023-01-10,139.55,NaN,NaN,NaN,NaN,NaN
2975,2023-01-11,139.97,NaN,NaN,NaN,NaN,NaN
2976,2023-01-12,139.40,NaN,NaN,NaN,NaN,NaN


In [17]:
tr.loc[15:30]

,datneg,preult,div_data_ex,div_data_pag,fr_de,fr_dc,fr_yf
15,2011-01-27,97.00,NaN,NaN,NaN,NaN,NaN
16,2011-01-28,96.60,NaN,NaN,NaN,NaN,NaN
17,2011-01-31,96.99,NaN,NaN,NaN,NaN,NaN
18,2011-02-01,96.99,0.675,NaN,1.006959,NaN,1.007008
19,2011-02-02,99.00,NaN,NaN,NaN,NaN,NaN
20,2011-02-03,98.50,NaN,NaN,NaN,NaN,NaN
21,2011-02-04,99.80,NaN,NaN,NaN,NaN,NaN
22,2011-02-07,97.00,NaN,NaN,NaN,NaN,NaN
23,2011-02-08,96.50,NaN,NaN,NaN,NaN,NaN
24,2011-02-09,97.00,NaN,NaN,NaN,NaN,NaN


In [18]:
# Preencher os valores NaN com 1 em fr_de, fr_dc e fr_yf para calcular o produtório
tr[["fr_de", "fr_dc", "fr_yf"]] = tr[["fr_de", "fr_dc", "fr_yf"]].fillna(1)
tr

,datneg,preult,div_data_ex,div_data_pag,fr_de,fr_dc,fr_yf
0,2011-01-04,100.00,NaN,NaN,1.0,1.000000,1.0
1,2011-01-05,99.70,NaN,NaN,1.0,1.000000,1.0
2,2011-01-06,99.70,NaN,NaN,1.0,1.000000,1.0
3,2011-01-07,99.50,NaN,NaN,1.0,1.000000,1.0
4,2011-01-10,95.50,NaN,NaN,1.0,1.000000,1.0
...,...,...,...,...,...,...,...
2973,2023-01-09,139.55,NaN,NaN,1.0,1.000000,1.0
2974,2023-01-10,139.55,NaN,NaN,1.0,1.000000,1.0
2975,2023-01-11,139.97,NaN,NaN,1.0,1.000000,1.0
2976,2023-01-12,139.40,NaN,NaN,1.0,1.000000,1.0


In [19]:
# Calcular o produtório dos fatores de reinvestimento
tr["p1"] = tr["fr_de"].cumprod()
tr["p2"] = tr["fr_dc"].cumprod()
tr["p3"] = tr["fr_yf"].cumprod()
# As colunas "fr_de", "fr_dc" e "fr_yf" não serão mais usadas -> remover
tr.drop(columns=["fr_de", "fr_dc", "fr_yf","div_data_ex", "div_data_pag"], inplace=True)
tr

,datneg,preult,p1,p2,p3
0,2011-01-04,100.00,1.000000,1.000000,1.000000
1,2011-01-05,99.70,1.000000,1.000000,1.000000
2,2011-01-06,99.70,1.000000,1.000000,1.000000
3,2011-01-07,99.50,1.000000,1.000000,1.000000
4,2011-01-10,95.50,1.000000,1.000000,1.000000
...,...,...,...,...,...
2973,2023-01-09,139.55,2.303694,2.288011,2.304006
2974,2023-01-10,139.55,2.303694,2.288011,2.304006
2975,2023-01-11,139.97,2.303694,2.288011,2.304006
2976,2023-01-12,139.40,2.303694,2.288011,2.304006


In [20]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["p1"],
        mode="lines",
        name="quantidades de cotas",
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Quantidade de Cotas do KNRI11",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Quantidade de Cotas",
)
fig.show()

In [21]:
print(tr.tail().to_markdown( tablefmt="psql", index=False, numalign="right", floatfmt=".4f"))

+---------------------+----------+--------+--------+--------+
| datneg              |   preult |     p1 |     p2 |     p3 |
|---------------------+----------+--------+--------+--------|
| 2023-01-09 00:00:00 | 139.5500 | 2.3037 | 2.2880 | 2.3040 |
| 2023-01-10 00:00:00 | 139.5500 | 2.3037 | 2.2880 | 2.3040 |
| 2023-01-11 00:00:00 | 139.9700 | 2.3037 | 2.2880 | 2.3040 |
| 2023-01-12 00:00:00 | 139.4000 | 2.3037 | 2.2880 | 2.3040 |
| 2023-01-13 00:00:00 | 138.8000 | 2.3037 | 2.3045 | 2.3040 |
+---------------------+----------+--------+--------+--------+


In [22]:
tr["preult_tr"] = tr["preult"] * tr["p1"]
tr

,datneg,preult,p1,p2,p3,preult_tr
0,2011-01-04,100.00,1.000000,1.000000,1.000000,100.000000
1,2011-01-05,99.70,1.000000,1.000000,1.000000,99.700000
2,2011-01-06,99.70,1.000000,1.000000,1.000000,99.700000
3,2011-01-07,99.50,1.000000,1.000000,1.000000,99.500000
4,2011-01-10,95.50,1.000000,1.000000,1.000000,95.500000
...,...,...,...,...,...,...
2973,2023-01-09,139.55,2.303694,2.288011,2.304006,321.480460
2974,2023-01-10,139.55,2.303694,2.288011,2.304006,321.480460
2975,2023-01-11,139.97,2.303694,2.288011,2.304006,322.448012
2976,2023-01-12,139.40,2.303694,2.288011,2.304006,321.134906


In [23]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["preult"],
        mode="lines",
        name="Cotação de Mercado",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["preult_tr"],
        mode="lines",
        name="Cotação Total Return",
        line=dict(color="#02878e"),
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="KNRI11<br>Cotação de Mercado vs Cotação Total Return",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Valor (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()